# Iridium Complex Oxygen Sensor Optimization

This notebook demonstrates using Folio for multi-objective optimization of luminescent
iridium complex oxygen sensors. We optimize sensor fabrication conditions to maximize
both sensitivity (Ksv) and linearity (R²) of the Stern-Volmer response.

**Stern-Volmer Relationship:**
$$\frac{I_0}{I} = 1 + K_{sv}[O_2]$$

Where:
- $I_0$ = luminescence intensity at 0% O₂
- $I$ = intensity at a given O₂ concentration
- $K_{sv}$ = Stern-Volmer constant (sensitivity)

**Optimization Objectives:**
1. Maximize $K_{sv}$ (higher sensitivity to oxygen)
2. Maximize $R^2$ (better linearity for reliable calibration)

In [ ]:
import sys
import tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, ".")

from folio.api import Folio
from folio.core.config import RecommenderConfig, TargetConfig
from folio.core.schema import InputSpec, OutputSpec

## Helper Functions

We define helper functions to compute Stern-Volmer parameters from intensity measurements.
In a real lab workflow, you would measure intensities and compute these metrics before
entering them into Folio.

In [ ]:
def compute_stern_volmer(I_0, I_5, I_10, I_15, I_20):
    """Compute Ksv and R² from intensity measurements.
    
    The Stern-Volmer equation: I₀/I = 1 + Ksv[O₂]
    """
    o2_conc = np.array([5.0, 10.0, 15.0, 20.0])
    intensities = np.array([I_5, I_10, I_15, I_20])
    
    # Compute I₀/I ratios
    sv_ratios = I_0 / intensities
    
    # Linear fit: I₀/I = 1 + Ksv*[O₂], slope = Ksv
    coeffs = np.polyfit(o2_conc, sv_ratios, 1)
    ksv = coeffs[0]
    
    # Compute R²
    y_pred = np.polyval(coeffs, o2_conc)
    ss_res = np.sum((sv_ratios - y_pred) ** 2)
    ss_tot = np.sum((sv_ratios - np.mean(sv_ratios)) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 1.0
    
    return ksv, r2

# Test with example data
ksv, r2 = compute_stern_volmer(12500, 9800, 7900, 6500, 5400)
print(f"Example: Ksv = {ksv:.4f}, R² = {r2:.4f}")

## Project Setup

We define the optimization space for sensor fabrication:

**Inputs (fabrication conditions):**
- Ir complex concentration (0.1–2.0 mM)
- PDMS film thickness (10–100 µm)
- Curing temperature (60–120 °C)
- Curing time (1–6 hours)

**Outputs (computed from intensity measurements):**
- Ksv: Stern-Volmer constant (sensitivity)
- R²: Coefficient of determination (linearity)

In [ ]:
# Create temporary database for demo
db_file = tempfile.NamedTemporaryFile(suffix=".db", delete=False)
folio = Folio(db_file.name)

# Define input parameters
inputs = [
    InputSpec(name="Ir_conc", type="continuous", bounds=(0.1, 2.0), units="mM"),
    InputSpec(name="thickness", type="continuous", bounds=(10.0, 100.0), units="µm"),
    InputSpec(name="cure_temp", type="continuous", bounds=(60.0, 120.0), units="°C"),
    InputSpec(name="cure_time", type="continuous", bounds=(1.0, 6.0), units="hours"),
]

# Define output measurements (Ksv and R² computed from raw intensities)
outputs = [
    OutputSpec(name="Ksv", units="1/%O2"),
    OutputSpec(name="R2"),
]

# Configure targets - both maximized
target_configs = [
    TargetConfig(objective="Ksv", objective_mode="maximize"),
    TargetConfig(objective="R2", objective_mode="maximize"),
]

# Reference point for multi-objective optimization
# Worst acceptable: Ksv < 0.01, R² < 0.9
reference_point = [0.01, 0.9]

# Create the project
folio.create_project(
    name="iridium_o2_sensor",
    inputs=inputs,
    outputs=outputs,
    target_configs=target_configs,
    reference_point=reference_point,
    recommender_config=RecommenderConfig(
        type="bayesian",
        surrogate="multitask_gp",
        mo_acquisition="nehvi",
        n_initial=5,
    ),
)

print("Project created: iridium_o2_sensor")
print(f"  Inputs: {[i.name for i in inputs]}")
print(f"  Outputs: Ksv (maximize), R² (maximize)")

---
## Day 1: Initial Screening

Starting sensor optimization project today. Goal is to find optimal PDMS film fabrication
conditions for our Ir(ppy)₃-based oxygen sensor. Will begin with some screening experiments
to understand the parameter space.

**Workflow:** Measure intensities at 0%, 5%, 10%, 15%, 20% O₂, then compute Ksv and R².

In [ ]:
# Get first suggestion
suggestion = folio.suggest("iridium_o2_sensor")[0]
print("First experiment suggestion:")
for key, value in suggestion.items():
    print(f"  {key}: {value:.2f}")

In [ ]:
# Experiment 1: Following BO suggestion
# Raw measurements: I_0=12500, I_5=9800, I_10=7900, I_15=6500, I_20=5400
ksv, r2 = compute_stern_volmer(12500, 9800, 7900, 6500, 5400)

folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 0.85, "thickness": 45.0, "cure_temp": 95.0, "cure_time": 3.5},
    outputs={"Ksv": ksv, "R2": r2},
    tag="screening",
    notes="First film. Good optical clarity. Slight edge curling during cure.",
)
print(f"Experiment 1: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 2: High concentration test
ksv, r2 = compute_stern_volmer(18200, 12100, 8900, 6800, 5500)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.8, "thickness": 30.0, "cure_temp": 80.0, "cure_time": 4.0},
    outputs={"Ksv": ksv, "R2": r2},
    tag="screening",
    notes="High [Ir] gives strong signal but film slightly yellow. Some aggregation visible under microscope.",
)
print(f"Experiment 2: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 3: Low concentration test  
ksv, r2 = compute_stern_volmer(4200, 3500, 3000, 2600, 2300)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 0.2, "thickness": 70.0, "cure_temp": 100.0, "cure_time": 2.5},
    outputs={"Ksv": ksv, "R2": r2},
    tag="screening",
    notes="Low [Ir] - weak signal, excellent film quality. Very uniform.",
)
print(f"Experiment 3: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 4: Thick film test
ksv, r2 = compute_stern_volmer(22000, 17500, 14200, 11800, 9900)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.0, "thickness": 90.0, "cure_temp": 70.0, "cure_time": 5.0},
    outputs={"Ksv": ksv, "R2": r2},
    tag="screening",
    notes="Thick film - very high I₀ but sluggish response. O₂ diffusion limited?",
)
print(f"Experiment 4: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 5: High temp cure
ksv, r2 = compute_stern_volmer(8500, 6200, 4700, 3700, 3000)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 0.6, "thickness": 50.0, "cure_temp": 115.0, "cure_time": 1.5},
    outputs={"Ksv": ksv, "R2": r2},
    tag="screening",
    notes="High temp cure - fast but film has micro-cracks. May affect long-term stability.",
)
print(f"Experiment 5: Ksv={ksv:.4f}, R²={r2:.4f}")
print("\nEnd of Day 1.")

### Day 1 Summary

Let's review our screening data before continuing tomorrow.

In [ ]:
observations = folio.get_observations("iridium_o2_sensor")

data = []
for obs in observations:
    data.append({
        "[Ir] mM": obs.inputs["Ir_conc"],
        "Thick µm": obs.inputs["thickness"],
        "Temp °C": obs.inputs["cure_temp"],
        "Time h": obs.inputs["cure_time"],
        "Ksv": f"{obs.outputs['Ksv']:.4f}",
        "R²": f"{obs.outputs['R2']:.4f}",
        "Notes": obs.notes[:35] + "..." if len(obs.notes) > 35 else obs.notes,
    })

df = pd.DataFrame(data)
print("Day 1 Screening Results:")
print(df.to_string(index=False))

---
## Day 2: Optimization Phase

Based on screening, mid-range [Ir] (0.5-1.2 mM) seems best. Thick films reduce sensitivity.
High cure temp causes cracking. Will focus optimization in these regions.

In [ ]:
suggestion = folio.suggest("iridium_o2_sensor")[0]
print("Day 2 - Suggestion 1:", {k: f"{v:.2f}" for k, v in suggestion.items()})

In [ ]:
# Experiment 6
ksv, r2 = compute_stern_volmer(14800, 10200, 7500, 5800, 4700)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.2, "thickness": 35.0, "cure_temp": 85.0, "cure_time": 3.0},
    outputs={"Ksv": ksv, "R2": r2},
    tag="optimization",
    notes="Good balance of signal and quenching. Film quality excellent.",
)
print(f"Experiment 6: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 7
ksv, r2 = compute_stern_volmer(11200, 8100, 6100, 4800, 3900)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 0.9, "thickness": 40.0, "cure_temp": 90.0, "cure_time": 4.0},
    outputs={"Ksv": ksv, "R2": r2},
    tag="optimization",
    notes="Very linear Stern-Volmer plot. Promising candidate.",
)
print(f"Experiment 7: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 8
ksv, r2 = compute_stern_volmer(13500, 8800, 6200, 4600, 3600)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.4, "thickness": 25.0, "cure_temp": 75.0, "cure_time": 4.5},
    outputs={"Ksv": ksv, "R2": r2},
    tag="optimization",
    notes="Thin film, high [Ir]. Strong quenching but slight non-linearity at high O₂.",
)
print(f"Experiment 8: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 9 - made a mistake, need to delete and re-enter
ksv_bad, r2_bad = compute_stern_volmer(9500, 7200, 5600, 4500, 370)  # Typo: should be 3700
print(f"Oops! R² = {r2_bad:.4f} looks wrong - I_20 was entered incorrectly")

# Correct calculation
ksv, r2 = compute_stern_volmer(9500, 7200, 5600, 4500, 3700)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 0.7, "thickness": 55.0, "cure_temp": 95.0, "cure_time": 2.0},
    outputs={"Ksv": ksv, "R2": r2},
    tag="optimization",
    notes="Moderate conditions. Decent results but not outstanding.",
)
print(f"Experiment 9 (corrected): Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiment 10
ksv, r2 = compute_stern_volmer(13200, 9400, 7000, 5400, 4300)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.1, "thickness": 42.0, "cure_temp": 88.0, "cure_time": 3.5},
    outputs={"Ksv": ksv, "R2": r2},
    tag="optimization",
    notes="Following BO suggestion closely. Excellent Stern-Volmer linearity!",
)
print(f"Experiment 10: Ksv={ksv:.4f}, R²={r2:.4f}")
print("\nEnd of Day 2.")

---
## Day 3: Fine-tuning and Replicates

Experiment 10 looked very promising. Today: explore nearby conditions and run replicates
of best performers.

In [ ]:
# Filter to see optimization experiments only
opt_obs = folio.get_observations("iridium_o2_sensor", tag="optimization")
print(f"Optimization experiments so far: {len(opt_obs)}")
for obs in opt_obs:
    print(f"  [Ir]={obs.inputs['Ir_conc']:.1f}, thick={obs.inputs['thickness']:.0f} -> Ksv={obs.outputs['Ksv']:.4f}, R²={obs.outputs['R2']:.4f}")

In [ ]:
suggestion = folio.suggest("iridium_o2_sensor")[0]
print("Day 3 - Suggestion:", {k: f"{v:.2f}" for k, v in suggestion.items()})

In [ ]:
# Experiments 11-12: Near-optimal region
ksv, r2 = compute_stern_volmer(12100, 8500, 6300, 4900, 3900)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.0, "thickness": 38.0, "cure_temp": 92.0, "cure_time": 3.2},
    outputs={"Ksv": ksv, "R2": r2},
    tag="optimization",
    notes="Near-optimal region exploration.",
)
print(f"Experiment 11: Ksv={ksv:.4f}, R²={r2:.4f}")

ksv, r2 = compute_stern_volmer(13800, 9700, 7200, 5500, 4400)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.15, "thickness": 40.0, "cure_temp": 90.0, "cure_time": 3.8},
    outputs={"Ksv": ksv, "R2": r2},
    tag="optimization",
    notes="Slightly higher [Ir]. Best Ksv so far!",
)
print(f"Experiment 12: Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiments 13-14: Replicates of best conditions
ksv, r2 = compute_stern_volmer(13050, 9300, 6950, 5350, 4250)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.1, "thickness": 42.0, "cure_temp": 88.0, "cure_time": 3.5},
    outputs={"Ksv": ksv, "R2": r2},
    tag="replicate",
    notes="Replicate of Exp 10. Results very reproducible!",
)
print(f"Experiment 13 (replicate): Ksv={ksv:.4f}, R²={r2:.4f}")

ksv, r2 = compute_stern_volmer(13350, 9500, 7100, 5450, 4350)
folio.add_observation(
    project_name="iridium_o2_sensor",
    inputs={"Ir_conc": 1.1, "thickness": 42.0, "cure_temp": 88.0, "cure_time": 3.5},
    outputs={"Ksv": ksv, "R2": r2},
    tag="replicate",
    notes="Replicate 2 of Exp 10. Consistent.",
)
print(f"Experiment 14 (replicate): Ksv={ksv:.4f}, R²={r2:.4f}")

In [ ]:
# Experiments 15-18: More optimization
for exp_data in [
    {"inputs": {"Ir_conc": 1.3, "thickness": 32.0, "cure_temp": 82.0, "cure_time": 4.2},
     "intensities": (14200, 9600, 6900, 5200, 4100),
     "notes": "Higher [Ir], thinner film. High Ksv but slightly lower R²."},
    {"inputs": {"Ir_conc": 0.95, "thickness": 45.0, "cure_temp": 85.0, "cure_time": 3.0},
     "intensities": (11500, 8300, 6200, 4850, 3900),
     "notes": "Slightly lower [Ir]. Good linearity."},
    {"inputs": {"Ir_conc": 1.05, "thickness": 38.0, "cure_temp": 93.0, "cure_time": 3.3},
     "intensities": (12600, 8800, 6500, 5000, 4000),
     "notes": "Exploring cure temp effect."},
    {"inputs": {"Ir_conc": 1.2, "thickness": 36.0, "cure_temp": 87.0, "cure_time": 3.6},
     "intensities": (14000, 9800, 7200, 5500, 4400),
     "notes": "Final optimization point. Excellent performance!"},
]:
    ksv, r2 = compute_stern_volmer(*exp_data["intensities"])
    folio.add_observation(
        project_name="iridium_o2_sensor",
        inputs=exp_data["inputs"],
        outputs={"Ksv": ksv, "R2": r2},
        tag="optimization",
        notes=exp_data["notes"],
    )
    print(f"  [Ir]={exp_data['inputs']['Ir_conc']}: Ksv={ksv:.4f}, R²={r2:.4f}")

print("\nEnd of Day 3.")

---
## Results Analysis

Let's analyze all our experiments and visualize the Pareto front.

In [ ]:
# Get all observations
all_obs = folio.get_observations("iridium_o2_sensor")
print(f"Total experiments: {len(all_obs)}")

results = []
for obs in all_obs:
    results.append({
        "Ir_conc": obs.inputs["Ir_conc"],
        "thickness": obs.inputs["thickness"],
        "cure_temp": obs.inputs["cure_temp"],
        "cure_time": obs.inputs["cure_time"],
        "Ksv": obs.outputs["Ksv"],
        "R2": obs.outputs["R2"],
        "tag": obs.tag,
    })

results_df = pd.DataFrame(results)
print("\nResults summary:")
print(results_df.to_string(index=False))

In [ ]:
# Plot Pareto front
fig, ax = plt.subplots(figsize=(8, 6))

colors = {"screening": "blue", "optimization": "green", "replicate": "red"}
for tag in colors:
    subset = results_df[results_df["tag"] == tag]
    ax.scatter(subset["Ksv"], subset["R2"], c=colors[tag], label=tag.capitalize(),
               s=100, alpha=0.7, edgecolors="black", linewidth=0.5)

ax.set_xlabel("Ksv (sensitivity)", fontsize=12)
ax.set_ylabel("R² (linearity)", fontsize=12)
ax.set_title("Multi-Objective Optimization: Ksv vs R²", fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Identify Pareto-optimal points
ksv_vals = results_df["Ksv"].values
r2_vals = results_df["R2"].values
pareto_mask = np.ones(len(results_df), dtype=bool)
for i in range(len(results_df)):
    for j in range(len(results_df)):
        if i != j:
            if ksv_vals[j] >= ksv_vals[i] and r2_vals[j] >= r2_vals[i]:
                if ksv_vals[j] > ksv_vals[i] or r2_vals[j] > r2_vals[i]:
                    pareto_mask[i] = False
                    break

pareto_df = results_df[pareto_mask].sort_values("Ksv")
ax.plot(pareto_df["Ksv"], pareto_df["R2"], "k--", alpha=0.6, linewidth=2)
ax.scatter(pareto_df["Ksv"], pareto_df["R2"], c="gold", s=200, marker="*",
           edgecolors="black", linewidth=1, zorder=10, label="Pareto optimal")
ax.legend()

plt.tight_layout()
plt.show()

print("\nPareto-optimal conditions:")
print(pareto_df[["Ir_conc", "thickness", "cure_temp", "cure_time", "Ksv", "R2"]].to_string(index=False))

## Conclusions

Through systematic Bayesian optimization with Folio, we identified optimal conditions for
iridium complex oxygen sensor fabrication:

**Optimal conditions (balanced Ksv and R²):**
- [Ir] concentration: ~1.1-1.2 mM
- Film thickness: ~36-42 µm
- Cure temperature: ~85-90 °C
- Cure time: ~3.5 hours

**Key findings:**
1. Higher [Ir] increases sensitivity (Ksv) but can reduce linearity at very high concentrations
2. Thinner films show better O₂ response (less diffusion limitation)
3. Moderate cure temperatures (85-92°C) give best film quality
4. Results are reproducible (low variance in replicates)

The Pareto front shows the tradeoff between sensitivity and linearity, allowing users
to select conditions based on their specific application requirements.

In [ ]:
# Cleanup
import os
os.unlink(db_file.name)
print("Demo complete. Temporary database cleaned up.")